## 銘柄からお金が流出してる状況を表現する方法はないかな？


### ヤッテミヨウ
1. PrevChangeにその日の出来高と終値を掛けるとか？

In [ ]:
import numpy as np

from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.morningstar import Fundamentals
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import Q500US, StaticSids, StaticAssets
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.factors import Returns

from quantopian.pipeline.classifiers.fundamentals import Sector


def make_pipeline(assets = None):
    
    if assets:
        base_universe = assets
    else:
        base_universe = QTradableStocksUS()
        
    close_price = USEquityPricing.close.latest
    vol = USEquityPricing.volume.latest

    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    prev_change = Returns(inputs = [USEquityPricing.close], window_length=2)
    financial_health_grade = Fundamentals.financial_health_grade.latest
    
    money_flow = prev_change * vol * close_price
    
    pipe = Pipeline(
        columns={
            'morningstar_industry_group_code':morningstar_industry_group_code, 
            'close_price':close_price, 
            'vol':vol,
            'money_flow':money_flow,
            "prev_change": prev_change,
            'financial_health_grade':financial_health_grade,
        },
        
        screen=base_universe,
    )
    return pipe 

pipeline_output = run_pipeline(make_pipeline(), '2019-12-01', '2020-06-18')
pipeline_output.head()

In [ ]:
x = pipeline_output.xs(symbols('DAL'), level=1) 
x["money_flow"].cumsum().plot()
x["close_price"].plot(secondary_y=True);



In [ ]:
asset_list = ["SPY", "GLD", "GDX"]
assets = StaticAssets(symbols(asset_list))

myassets_output = run_pipeline(make_pipeline(assets), '2019-01-01', '2020-06-18')
myassets_output.head()

In [ ]:
y = myassets_output.xs(symbols('GDX'), level=1) 
y["money_flow"].plot()
y["close_price"].plot(secondary_y=True);


In [ ]:

airlines = pipeline_output[pipeline_output["morningstar_industry_group_code"]==31080]
df = airlines["money_flow"].unstack()
close_price = airlines["close_price"].unstack()



In [ ]:
df.tail()

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib 

fig = plt.figure(figsize=(50,50))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)


num = len(df.columns)
xs = 10
ys = int(num/xs)+1
for i, c in enumerate(df.columns):
    x = i % xs
    y = int(i/xs)+ys
    ax = fig.add_subplot(xs, ys, i+1)
    ax.set_title(c.symbol)
    ax.plot(df[c].cumsum())
    ax2 = ax.twinx()
    ax2.plot(close_price[c], c="green")
    ax2.grid(b=False)
    